# Lab HW 2

# **Questions**


In [ ]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
import re
from collections import defaultdict, Counter
import string
nltk.download('punkt')

#**Q1) Write a program that parses a given word into its root form and affixes (prefixes and suffixes). The program should output the morphemes.**
Example Input: "unhappiness"

Example Output: ["un-", "happy", "-ness"]

- Here, We have to handle words like happiness, craziness, competition whose root forms can not be obtained just by removing suffix.

In [ ]:
def give_morphemes(word):
    prefix = ['un', 're', 'in', 'dis', 'im', 'ir', 'il', ]
    suffix = ['ed', 'ing', 'ly', 'ness', 's', 'er', 'able', 'ment', 'ition']

    base_word = word; ans=[]; t1= None

    # Checking for prefixes
    for pref in prefix:
        if word.startswith(pref):
            base_word = word[len(pref):]
            ans.append( pref + "-")
            break

    # Checking for suffixes
    for suf in suffix:
        if base_word.endswith(suf):
            base_word = base_word[:-len(suf)]
            t1= suf
            break

    # Handle special cases like 'happi' to 'happy'
    if base_word.endswith('i') and word.endswith('ness'):
        base_word = base_word[:-1] + 'y'

    if word.endswith('ition'):
      base_word = base_word + "e"

    ans.append(base_word)
    if t1 is not None:
      ans.append("-"+t1)
    return ans

# Example usage
print( give_morphemes("unhappiness") )
print(give_morphemes("retry") )
print( give_morphemes("tryre") )
print(give_morphemes("craziness"))
print(give_morphemes('competition'))

#**Q2) Write a program that identifies the derivational morphemes in a given sentence and classifies each word into its base form and derivational affixes.**

Example:
Input Sentence: "The artistically gifted boy performed exceptionally in the
competition."

Output:

    "word": "artistically",
    "base_form": "art",
    "derivational_affixes": ["-ist", "-ic", "-ally"],
  
    "word": "gifted",
    "base_form": "gift",
    "derivational_affixes": ["-ed"],
  
    "word": "performed",
    "base_form": "perform",
    "derivational_affixes": ["-ed"],
  
    "word": "exceptionally",
    "base_form": "except",
    "derivational_affixes": ["-ion", "-al", "-ly"],
  
    "word": "competition",
    "base_form": "compete",
    "derivational_affixes": ["-ition"]


- This is a custom implemented method, here we first separate the sentence using " " and then apply the method `get_derivational_morphemes`.
- For the method, we iterate from the back of the word and separate out the smallest affix which we can find in the word and start out again from new end of the word.

In [ ]:

# Derivational morphemes we are going to check
derivational_affixes = ['ness','ally','ful','less','ment','er','est','ing','ed', 'ize','ation','al','ist','ic', 'ed', 'ion', 'ition']

def get_derivational_morphemes(word):

    affix = []
    i= len(word)-1; j=len(word)
    while(i>-1):
      if word[i:j] in derivational_affixes:

        if(word[i:j]=='ion' and word[i-2:i]=='it'): # handling the case for -ition
          affix.append("-"+word[i-2:j]);
          i=i-2;
        else:
          affix.append("-"+word[i:j]);

        j=i

      i-=1

    return word[:j], affix

def label_morphemes(sen):

    results = []
    doc= sen.split(); doc[-1].strip('.')

    for word in doc:

        base_form, affix = get_derivational_morphemes(word.lower())
        if len(affix)>=1:
            results.append({
                'word': word,
                'base_form': base_form,
                'affix': affix[::-1]
            })

    return results

sen = "The artistically gifted boy performed exceptionally in the competition."
results = label_morphemes(sen)

for result in results:
    print(f"Word: {result['word']}, Base Form: {result['base_form']}, derivatonal_affixes: {result['affix']}")


#**Q3) Implement a word segmentation program that can handle ambiguous cases where multiple segmentations are possible. The program should use a probabilistic model to choose the most likely segmentation.**
Example Input: "therapistfinder"

Example Output: ["therapist", "finder"] or ["the", "rapist", "finder"] (depending on context)

- A dynammic programming based approach is used to find the optimum word.
- Optimum word implies here the the word which has the highest probability to come alone.
- This is being done by giving the data for frequency of each word. I have taken frequencies to be random here.

In [ ]:
import math

class mySegmenter:

  def __init__(self, sample_text):
      self.probability = self._build_probab_table(sample_text)

  def _build_probab_table(self, sample_text):
      n = sum(sample_text.values() )
      return { w: math.log(coun / n) for w, coun in sample_text.items()}

  def segment_word(self, test_text):

      sz:int = len(test_text)
      dp_table = [None] * (sz + 1);  dp_table[0] = (0, [])

      for i in range(1, sz + 1):
        options = []
        for j in range(i):
          w = test_text[j:i]
          if w in self.probability:
            cur_prob = self.probability[w]
            if dp_table[j]!=None:
              options.append((dp_table[j][0] + cur_prob, dp_table[j][1] + [w]))

        if(len(options)!=0):
          dp_table[i] = max(options, key=lambda x: x[0]) # finding max values based on higher probability

      return dp_table[sz][1] if dp_table[sz] else [test_text]

# The given corpus is a dict with key as the word and its value as the number of occurences of it
data = [{ "the": 1423,  "therapist": 23,  "rapist": 100,  "finder": 24, "ist": 201, "istfinder": 3, "ther":4, "find": 10  },
 { "it":  1524, "grey": 1000, "isgrey":  10, "is": 4000, "ey": 1000 , "itis":1 }
]

seg1 = mySegmenter(data[0])
seg2 = mySegmenter(data[1])
sample1 = "therapistfinder";  sample2= "itisgrey"

output1 = seg1.segment_word(sample1)
output2= seg2.segment_word(sample2)
print("Segmented Words:", output1)
print("Segmented Words:", output2)


#**Q4) Write a program that uses a bigram language model to predict the next word in a given context. The program should consider the previous word to predict the next word with the highest probability.**
Task: Train a bigram model on a text corpus and predict the next word for a given input word.

**Example:**
Once upon a time there was a brave knight.

The knight was known for his bravery and courage.

He fought many battles and won the hearts of the people.

The people of the kingdom loved the knight.

One day, the knight embarked on a journey to defeat a dragon.

The dragon was terrorizing the villages in the kingdom.

After a long and fierce battle, the knight defeated the dragon.

The people celebrated the victory of the knight.

The knight became a legend in the kingdom.


**what is most probable next word predicted by the modal?**

"The knight"

"The people"

"The dragon"

"A brave"

"The kingdom"

**Expected Predictions:**
For "The knight", the model might predict "was".

For "The people", it might predict "of" or "celebrated".

For "The dragon", it could predict "was" or "defeated".

For "A brave", it might predict "knight".

For "The kingdom", it could predict "loved" or "celebrated".

- I believe that the example was misplaced here as the example discusses a trigram model whereas a bigram model was used for prediction of the next word.

In [ ]:

test_text = """
Once upon a time there was a brave knight. The knight was known for his bravery and courage.
He fought many battles and won the hearts of the people. The people of the kingdom loved the knight.
One day, the knight embarked on a journey to defeat a dragon. The dragon was terrorizing the villages in the kingdom.
After a long and fierce battle, the knight defeated the dragon. The people celebrated the victory of the knight.
The knight became a legend in the kingdom.
"""

tokens = nltk.word_tokenize(test_text.lower())

bigrams = list( ngrams(tokens, 2))

freq_grams = defaultdict(Counter)
for w1, w2 in bigrams:
    freq_grams[w1][w2] += 1

def predict_next( w1 ,freq_grams):
    key = w1
    if key in freq_grams:
        options = freq_grams[key].most_common()

        options = [(w, coun) for w, coun in options if w not in string.punctuation]
        if options:
            return options[0][0]
        else:
            return "Cannot predict further word"
    else:
        return "Cannot predict further word"


print(predict_next("the", freq_grams))
print(predict_next("and", freq_grams))
print(predict_next("a", freq_grams))

#**Q5) Write a program that implements an N-gram (e.g., trigram) language model. The program should calculate the probability of a given sequence of words.**
Task: Implement a trigram model using a provided text corpus. Calculate the probability of a given sentence.

Corpus:

I am a human.

I am not a stone.

I I live in Mumbai.

Check the probability of " I I am not "



- Below, I have a tri-gram model.

In [ ]:
test_text2= """
I am a human.
I am not a stone.
I I live in Mumbai.
"""

# Tokenize the corpus
tokens = nltk.word_tokenize(test_text.lower())

# Generate 3-grams and 2-grams
thregrams = list(ngrams(tokens, 3))
twograms = list(ngrams(tokens, 2))

# Count the frequencies of 4-grams and 3-grams
freq_trigrams = defaultdict(Counter)
freq_bigrams = Counter()

for w1, w2, w3 in thregrams:
    freq_trigrams[(w1, w2)][w3] += 1

for w1, w2 in twograms:
    freq_bigrams[(w1, w2)] += 1

def find_probability(sentence, trigram_freq, bigram_freq):
    tokens = nltk.word_tokenize(sentence.lower())
    trigrams = list(ngrams(tokens, 3))

    prob = 1.0
    for w1, w2, w3 in trigrams:

        bigram = (w1, w2)
        coun_bigram = bigram_freq[bigram]
        coun_trigram = trigram_freq[bigram][w3]

        # applying laplacian smoothing
        vocab_size = len( set(tokens) )
        prob *= ( coun_trigram + 1 ) / ( coun_bigram + vocab_size )

    return prob

sen = "I I am not"
probability = find_probability( sen, freq_trigrams, freq_bigrams )

print(f'Probability of "{sen}": {probability}')


#**Q6) Write a program that applies smoothing techniques (e.g., Laplace, Good-Turing) to a bigram language model. Compare the performance of the model with and without smoothing.**

Corpus:

I love natural language processing.

Language models are great.

I love machine learning.

Machine learning is fun.

Natural language processing is a complex field.


In [ ]:
# Corpus
test_text3 = """
I love natural language processing.
Language models are great.
I love machine learning.
Machine learning is fun.
Natural language processing is a complex field.
"""

tokens = nltk.word_tokenize(test_text3.lower())
bigrams = list(ngrams(tokens, 2))

freq_bigrams = Counter(bigrams)
freq_onegrams = Counter(tokens)

def without_smoothing( w1, w2, freq_bigrams, freq_onegram ):

    bigram = ( w1, w2)
    coun_bigram = freq_bigrams[bigram ]
    coun_word = freq_onegram[ w1 ]

    if coun_word == 0:
        return 0

    return coun_bigram / coun_word

def laplace_smoothing(w1, w2, freq_bigrams, freq_onegram ):

    bigram = (w1, w2)
    n = len(set(tokens))
    coun_bigram = freq_bigrams[ bigram ]
    coun_word = freq_onegram[ w1 ]

    return (coun_bigram + 1) / (coun_word + n)

def sentence_probab(sentence, bigram_freqs, word_freqs, choice):

    tokens = nltk.word_tokenize(sentence.lower())
    bigrams = list(ngrams(tokens, 2))

    prob = 1.0
    if(choice==0):
      for w1, w2 in bigrams:
        prob *= without_smoothing(w1, w2, bigram_freqs, word_freqs)

    elif(choice==1):
      for w1, w2 in bigrams:
        prob *= laplace_smoothing(w1, w2, freq_bigrams, freq_onegrams)

    return prob

sentence = "I love machine processing"

unsmoothed_prob_value = sentence_probab(sentence, freq_bigrams, freq_onegrams, 0)
print(f'Unsmoothed probability of "{sentence}": {unsmoothed_prob_value}')

laplace_prob_value = sentence_probab(sentence, freq_bigrams, freq_onegrams, 1)
print(f'Laplace smoothed probability of "{sentence}": {laplace_prob_value}')


#**Q7) Write a program that implements a simple rule-based POS tagger. The program should take a sentence as input and assign POS tags based on predefined rules. For instance, you might define rules such as:**

Words ending in "ed" are tagged as verbs (e.g., "walked" → VBD).

Words ending in "ly" are tagged as adverbs (e.g., "quickly" → RB).
Example:

Input: "The cat quickly jumped over the lazy dog."

Output: [('The', 'DT'), ('cat', 'NN'), ('quickly', 'RB'), ('jumped', 'VBD'), ('over', 'IN'), ('the', 'DT'), ('lazy', 'JJ'), ('dog', 'NN')]

In [ ]:
def Tagging_Word(word):

    if word.lower() in {'an', 'the', 'a'}:
        return 'DT'
    elif word.lower() in {'by', 'with', 'for', 'of', 'in', 'on', 'at'}:
        return 'IN'
    elif word.lower() in {'is', 'am', 'are', 'was', 'were', 'shall', 'has', 'have', 'had'}:
        return 'VB'
    elif word in {'.', '-', '!', '?', ';', ',', ':'}:
        return 'PUNCT'
    elif word.endswith('ed'):
        return 'VBD'
    elif word.endswith('ly'):
        return 'RB'
    elif word.endswith('ing'):
        return 'VBG'
    elif word.endswith('s') and not word.endswith('ss'):
        return 'VBZ'
    elif word.endswith('ness') or word.endswith('ity'):
        return 'NN'
    elif word.endswith('er') or word.endswith('est') or ( word.endswith('y') and ( word[:-1] == "cloud" or word[:-1]=="malt" or word[:-1]=="mess" ) ):
        return 'JJ'
    elif word.endswith('ion') or word.endswith('ment'):
        return 'NN'
    else:
        return 'NN'

def tag_sentence(sen):

    words = word_tokenize(sen)
    tagged_words = [(word, Tagging_Word(word)) for word in words]

    return tagged_words

sen1 = "The cat quickly jumped over the lazy dog."
sen2= "It was a cloudy day in the morning."

print( tag_sentence(sen1) )
print( tag_sentence(sen2) )

The inference here is that it is very difficult to classify words into different POS as there could be many more exceptions like "cloudy" and "day".